<img src="./ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

<img src="./INPUT_OUTPUT.png" width=700px>




## Agent One is a code expert. Agent Two is an expert code reviewer.

We generate a response with our first query using a system prompt to create code.

We then pass the output into another function that acts as a reviewer to produce the next version of the code.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-1WUVgv...
GROQ_API_KEY exists and begins gsk_11hFN1EMfj...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)
# Output below...

**API Request with Requests Library**
=====================================

Below is a Python implementation of requesting an API using the Requests library.

### Prerequisites

* Python 3.6+
* `requests` library installed

### Code

```python
import requests
import json

def make_api_request(url, method='GET', headers=None, params=None, data=None):
    """
    Makes an API request using the Requests library.

    Args:
    - url (str): The URL of the API endpoint.
    - method (str): The HTTP method to use (e.g. GET, POST, PUT, DELETE). Defaults to 'GET'.
    - headers (dict): A dictionary of HTTP headers to include in the request.
    - params (dict): A dictionary of query parameters to include in the request.
    - data (dict): A dictionary of data to include in the request body.

    Returns:
    - response (requests.Response): The response from the API.
    """
    if method.upper() == 'GET':
        response = requests.get(url, headers=headers, params=params)
    elif method.upper() == 'POST':
        response = requests.post(url, headers=headers, params=params, json=data)
    elif method.upper() == 'PUT':
        response = requests.put(url, headers=headers, params=params, json=data)
    elif method.upper() == 'DELETE':
        response = requests.delete(url, headers=headers, params=params)
    else:
        raise ValueError("Invalid method. Supported methods are GET, POST, PUT, and DELETE.")

    return response

def main():
    # Example usage
    url = 'https://jsonplaceholder.typicode.com/todos/1'
    response = make_api_request(url)

    if response.status_code == 200:
        print(f"Response status code: {response.status_code}")
        print(f"Response content: {response.json()}")
    else:
        print(f"Error: {response.status_code}")

if __name__ == "__main__":
    main()
```

### Explanation

*   The `make_api_request` function takes in the API URL, HTTP method, headers, query parameters, and data as arguments.
*   It uses the `requests` library to make the API request based on the provided method and returns the response.
*   In the `main` function, an example usage of the `make_api_request` function is shown.
*   The response status code and content are printed if the request is successful.

### Usage

1.  Replace the `url` variable in the `main` function with the API endpoint you want to request.
2.  Modify the `make_api_request` function call in the `main` function to pass in the required headers, query parameters, and data for your API request.
3.  Run the script using Python (e.g., `python api_request.py`).

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT and we change the system prompt or what we want it to do along with the output from the previous query.


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)
# Critique displayed below...

**Code Review and Recommendations**
=====================================

The provided code is a nice implementation of making API requests using the Requests library in Python. However, there are a few areas that can be improved for better readability, maintainability, and robustness.

### 1. Error Handling

The code only checks for the status code of the response. It does not handle any exceptions that might occur during the request, such as connection errors or timeouts. It is a good practice to catch such exceptions and handle them accordingly.

```python
try:
    response = make_api_request(url)
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")
```

### 2. Response Validation

The code assumes that the response will always contain a JSON payload. However, this might not be the case for all APIs. The code should check the content type of the response before trying to parse it as JSON.

```python
if 'application/json' in response.headers['Content-Type']:
    print(f"Response content: {response.json()}")
else:
    print(f"Response content: {response.text}")
```

### 3. Type Hints

The function `make_api_request` does not use type hints for its parameters and return type. Adding type hints can improve the readability and maintainability of the code.

```python
def make_api_request(url: str, method: str = 'GET', headers: dict = None, params: dict = None, data: dict = None) -> requests.Response:
```

### 4. Method Enumeration

The code uses strings to represent the HTTP methods. This can be improved by using an enumeration instead. Python's `enum` module can be used for this purpose.

```python
from enum import Enum

class HttpMethod(Enum):
    GET = 'GET'
    POST = 'POST'
    PUT = 'PUT'
    DELETE = 'DELETE'

def make_api_request(url: str, method: HttpMethod = HttpMethod.GET, headers: dict = None, params: dict = None, data: dict = None) -> requests.Response:
```

### 5. Redundant Code

The code has redundant `if-elif-else` statements for handling different HTTP methods. This can be improved by using a dictionary to map the HTTP methods to their corresponding functions.

```python
def make_api_request(url: str, method: HttpMethod = HttpMethod.GET, headers: dict = None, params: dict = None, data: dict = None) -> requests.Response:
    methods = {
        HttpMethod.GET: requests.get,
        HttpMethod.POST: requests.post,
        HttpMethod.PUT: requests.put,
        HttpMethod.DELETE: requests.delete
    }

    if method in methods:
        response = methods[method](url, headers=headers, params=params, json=data)
    else:
        raise ValueError("Invalid method")
```

### 6. Code Refactoring

The code can be refactored to reduce repetition and improve readability.

```python
import requests
import json
from enum import Enum

class HttpMethod(Enum):
    GET = 'GET'
    POST = 'POST'
    PUT = 'PUT'
    DELETE = 'DELETE'

def make_api_request(url: str, method: HttpMethod = HttpMethod.GET, headers: dict = None, params: dict = None, data: dict = None) -> requests.Response:
    methods = {
        HttpMethod.GET: requests.get,
        HttpMethod.POST: requests.post,
        HttpMethod.PUT: requests.put,
        HttpMethod.DELETE: requests.delete
    }

    try:
        if method in methods:
            response = methods[method](url, headers=headers, params=params, json=data)
            response.raise_for_status()  # Raise an exception for bad status codes
            return response
        else:
            raise ValueError("Invalid method")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

def main():
    url = 'https://jsonplaceholder.typicode.com/todos/1'
    response = make_api_request(url)

    if response:
        if 'application/json' in response.headers['Content-Type']:
            print(f"Response content: {response.json()}")
        else:
            print(f"Response content: {response.text}")

if __name__ == "__main__":
    main()
```

This refactored code improves the readability and maintainability of the original code. It also adds better error handling and reduces repetition.

Add CRITIQUE to chat...

Notice how we are appending previous responses so that the next SYSTEM MESSAGE has history or context.


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)
# Response to critique displayed below...

The provided review and recommendations are excellent and address several key areas for improvement in the original code. Here's a further refactored version of the code that incorporates all the suggestions and adds a few more improvements for better readability, maintainability, and robustness.

```python
import requests
import json
from enum import Enum

class HttpMethod(Enum):
    GET = 'GET'
    POST = 'POST'
    PUT = 'PUT'
    DELETE = 'DELETE'

class ApiResponse:
    def __init__(self, response: requests.Response):
        self.response = response
        self.status_code = response.status_code
        self.headers = response.headers
        self.content_type = self.headers.get('Content-Type')

    def get_json(self):
        if 'application/json' in self.content_type:
            return self.response.json()
        return None

    def get_text(self):
        return self.response.text

def make_api_request(url: str, method: HttpMethod = HttpMethod.GET, headers: dict = None, params: dict = None, data: dict = None) -> ApiResponse:
    methods = {
        HttpMethod.GET: requests.get,
        HttpMethod.POST: requests.post,
        HttpMethod.PUT: requests.put,
        HttpMethod.DELETE: requests.delete
    }

    try:
        if method in methods:
            response = methods[method](url, headers=headers, params=params, json=data)
            response.raise_for_status()  # Raise an exception for bad status codes
            return ApiResponse(response)
        else:
            raise ValueError("Invalid method")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def main():
    url = 'https://jsonplaceholder.typicode.com/todos/1'
    response = make_api_request(url)

    if response:
        print(f"Status Code: {response.status_code}")
        json_response = response.get_json()
        if json_response:
            print(f"JSON Response: {json_response}")
        else:
            print(f"Text Response: {response.get_text()}")

if __name__ == "__main__":
    main()
```

This refactored code includes the following improvements:

1.  **Added a separate `ApiResponse` class**: This class encapsulates the response from the API, providing a clear and organized way to access the response's status code, headers, and content.
2.  **Improved error handling**: The code now handles exceptions that may occur during the API request, providing a clear error message and returning `None` in case of an error.
3.  **Added `get_json` and `get_text` methods**: These methods allow you to easily retrieve the response as JSON or text, depending on the content type of the response.
4.  **Simplified the `main` function**: The `main` function now only needs to call `make_api_request` and handle the response, making it easier to read and understand.
5.  **Improved code organization**: The code is now more modular, with separate classes and functions for handling API responses and making API requests. This makes it easier to maintain and extend the code.

Overall, this refactored code provides a robust, maintainable, and readable solution for making API requests and handling responses in Python.

<br>

### We can of course make this a Class...
